# WIP: Back-test

This example is still WIP.
* Load Data
* Back Test - the Basic Idea

In [1]:
from grandma_stock_valuation import FileLogger, loadPacakgeData
from grandma_stock_valuation import batchValuation, addCashPortfolio, getCorrelationWeight, allocatePortfolio

# Refer to example_0_FileLogger.ipynb for details of the FileLogger.
logger = FileLogger()
logPrint = logger.logPandas

### Load Data

For this example, I will use the stored package data.

To query data from Yahoo, please refer to *example_1_yahoo_data_loader.ipynb*.

In [2]:
d_instrument_data, d_instrument = loadPacakgeData(verbose=2)

d_instrument

EEMA data contains 2530 rows, 2530 dates from 2012-02-09 to 2022-02-28.
IEV data contains 5431 rows, 5431 dates from 2000-07-28 to 2022-02-28.
IVV data contains 5479 rows, 5479 dates from 2000-05-19 to 2022-02-28.
VPL data contains 4273 rows, 4273 dates from 2005-03-10 to 2022-02-28.


{'IVV': 'SP500',
 'VPL': 'Developed Asia-Pacific',
 'IEV': 'Europe',
 'EEMA': 'Emerging Asia'}

In [3]:
#### TEMP ####
l_select_inst = ['EEMA', 'VPL'] # , 'IVV', 

d_instrument_data = {k:v for k,v in d_instrument_data.items() if k in l_select_inst}

In [4]:
logPrint("Keys of d_instrument_data:", str(d_instrument_data.keys()))

#logPrint("IVV (SP500 ETF):", d_instrument_data['IVV'].head())

2022-03-12 02:34:28,666 INFO Keys of d_instrument_data: dict_keys(['EEMA', 'VPL'])


### Back Test - the Basic Idea

WIP
* back test is to understand how your strategy behaves under various scenanios, not to prove your strategy can beat xxx.
* most investment products are not built to beat xxx.
* historical performance cannot represent future performance.


### Back Test - WIP

In [5]:
from grandma_stock_valuation.back_test import GrandmaBackTester

backtester = GrandmaBackTester(
    backtest_years=10,
    adjust_freq_months=3,
    init_parameters={'recent_months':0, 'train_years':10, 'min_train_years':5, 'date_end':None},
    fit_parameters={'price_col':'close_adj', 'log':True, 'n_std':1.5},
    valuate_parameters={'min_annual_return':0.01},
    allocation_parameters={'transformation':'sigmoid', 'scale':1},
    with_cash=False,
    with_correlation_weights=True,
    verbose=1,
    printfunc=logPrint
)

backtester.runBackTest(d_instrument_data)

backtester.df_average_value


2022-03-12 02:34:28,743 INFO To backtest 10 years, from 2012-03-01 to 2022-02-28
2022-03-12 02:34:28,745 INFO EEMA's start date 2012-02-09 is beyond 2007-03-01 for full back-test.
2022-03-12 02:34:29,194 INFO Adjust portfolio on 2012-06-01, total value = 0.000000
2022-03-12 02:34:29,605 INFO Adjust portfolio on 2012-09-04, total value = 0.000000
2022-03-12 02:34:29,990 INFO Adjust portfolio on 2012-12-03, total value = 0.000000
2022-03-12 02:34:30,378 INFO Adjust portfolio on 2013-03-01, total value = 0.000000
2022-03-12 02:34:30,789 INFO Adjust portfolio on 2013-06-03, total value = 0.000000
2022-03-12 02:34:31,183 INFO Adjust portfolio on 2013-09-03, total value = 0.000000
2022-03-12 02:34:31,586 INFO Adjust portfolio on 2013-12-02, total value = 0.000000
2022-03-12 02:34:31,967 INFO Adjust portfolio on 2014-03-03, total value = 0.000000
2022-03-12 02:34:32,375 INFO Adjust portfolio on 2014-06-02, total value = 0.000000
2022-03-12 02:34:32,760 INFO Adjust portfolio on 2014-09-02, tot

,ticker,avg_pct_allocation
0,EEMA,NaN
1,VPL,NaN


In [6]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

#import plotly.io as pio
#pio.renderers.default = "notebook_connected"

In [7]:
# prepare hover text of portfolio adjustments
df_adjs = backtester.df_adjustments[['date', 'ticker', 'train_years', 'annualized_return', 'over_value_range', 'over_value_years', 'weight', 'portfolio_allocation']].copy()

df_adjs['train_years'] = df_adjs['train_years'].astype(float).round(1).astype(str)
df_adjs['annualized_return'] = (100*df_adjs['annualized_return'].astype(float)).round(1).astype(str)
df_adjs['over_value_range'] = (100*df_adjs['over_value_range'].astype(float)).round(1).astype(str)
df_adjs['over_value_years'] = df_adjs['over_value_years'].astype(float).round(2).astype(str)
df_adjs['weight'] = df_adjs['weight'].astype(float).round(2).astype(str)
df_adjs['portfolio_allocation'] = (100*df_adjs['portfolio_allocation'].astype(float)).round(1).astype(str)

df_adjs['adj_hover_text'] = ''

index_train = df_adjs['train_years']!='nan'
df_adjs['train_text'] = df_adjs['annualized_return'] + '% ann. growth in ' + df_adjs['train_years'] + 'yrs'
df_adjs['train_text'] = df_adjs['train_text'] + '<br>valuation = ' + df_adjs['over_value_range'] + '%'
df_adjs.loc[index_train, 'adj_hover_text'] = df_adjs['train_text'][index_train] + '<br>'

index_value = df_adjs['over_value_years']!='nan'
df_adjs['value_text'] = 'weight = ' + df_adjs['weight'] + '; valuation = ' + df_adjs['over_value_years'] + ' yrs'
df_adjs.loc[index_value, 'adj_hover_text'] = (df_adjs['adj_hover_text'] + df_adjs['value_text'])[index_value] + '<br>'

df_adjs['adj_hover_text'] = df_adjs['adj_hover_text'] + df_adjs['portfolio_allocation']  + '% Portfolio'


index_cash = df_adjs['ticker']=='cash'
df_adjs.loc[index_cash, 'adj_hover_text'] = df_adjs['adj_hover_text'][index_cash].str.replace('Portfolio', 'Cash')
df_adjs = df_adjs[['date','ticker','adj_hover_text']]

df_adjs.head()

,date,ticker,adj_hover_text
0,2012-03-01,EEMA,0.0% Portfolio
1,2012-03-01,VPL,-0.1% ann. growth in 7.0yrs<br>valuation = 4.2...
2,2012-06-01,EEMA,0.0% Portfolio
3,2012-06-01,VPL,-0.2% ann. growth in 7.2yrs<br>valuation = -10...
4,2012-09-04,EEMA,0.0% Portfolio


In [8]:
df_daily_portfolio_pct = backtester.df_portfolio[['date','ticker','current_portfolio_pct']]

# growth of Grandma
df_grandma_growth = backtester.df_total_value.melt(id_vars='date', var_name='ticker', value_name='growth')
df_grandma_growth['ticker'] = 'cash' # to merge with cash % for hover text display
df_grandma_growth = df_grandma_growth.merge(df_daily_portfolio_pct, 'left', ['date','ticker']).fillna(0)
df_grandma_growth.rename(columns={'current_portfolio_pct':'hover_text'}, inplace=True)
df_grandma_growth['hover_text'] = (100*df_grandma_growth['hover_text'].astype(float)).round(1).astype(str) + '% Cash'

# growth of each instrument
df_instrument_growth = backtester._df_instrument_prices.iloc[backtester._index_start:].copy()
d_first_prices = df_instrument_growth.apply(lambda se: se.dropna().iloc[0])

df_instrument_growth = df_instrument_growth.apply(lambda se: se/d_first_prices[se.name])
df_instrument_growth.columns = [c.replace('price_', '') for c in df_instrument_growth.columns]
df_instrument_growth = df_instrument_growth.reset_index()
df_instrument_growth = df_instrument_growth.melt(id_vars='date', var_name='ticker', value_name='growth')

df_instrument_growth = df_instrument_growth.merge(df_daily_portfolio_pct, 'left', ['date','ticker']).fillna(0)
df_instrument_growth.rename(columns={'current_portfolio_pct':'hover_text'}, inplace=True)
df_instrument_growth['hover_text'] = (100*df_instrument_growth['hover_text'].astype(float)).round(1).astype(str) + '% Portfolio'

# combine together for plotting
df_instrument_growth = pd.concat([df_grandma_growth, df_instrument_growth], ignore_index=True) # NOTE: sequence matters here

# change hover text of portfolio adjustment dates
df_instrument_growth = df_instrument_growth.merge(df_adjs, 'left', ['date', 'ticker'])
index_adj = df_instrument_growth['adj_hover_text'].notnull()
df_instrument_growth.loc[index_adj, 'hover_text'] = df_instrument_growth['adj_hover_text'][index_adj]
df_instrument_growth.drop(columns='adj_hover_text', inplace=True)

index_grandma = df_instrument_growth['ticker'] == 'cash'
df_instrument_growth.loc[index_grandma, 'ticker'] = 'Grandma'

df_instrument_growth.head(3)

,date,ticker,growth,hover_text
0,2012-03-01,Grandma,1.0,0.0% Cash
1,2012-03-02,Grandma,0.0,0.0% Cash
2,2012-03-05,Grandma,0.0,0.0% Cash


In [9]:
# plot the chart
fig = px.line(df_instrument_growth, x='date', y='growth',
              color='ticker', color_discrete_map={'Grandma':'white'},
              custom_data = ['hover_text'],
              )

fig.update_traces(hovertemplate='%{y:.2f}<br>%{customdata}')
fig.update_traces(line=dict(width=0.75))
fig['data'][0]['line']['width'] = 1.5

fig.update_layout(template='plotly_dark', title='Back-test',
                  xaxis_title='date', yaxis_title='growth',
                  hovermode='x unified',
                  xaxis_hoverformat='%Y-%m-%d')

index_adj_scatter = df_grandma_growth['date'].isin(set(backtester.df_adjustments['date']))
df_adj_scatter = df_grandma_growth[index_adj_scatter][['date','growth']]

fig.add_trace(go.Scatter(x=df_adj_scatter['date'], y=df_adj_scatter['growth'], mode='markers', showlegend=False,
                         marker={'color':'yellow', 'size':8, 'symbol':'diamond-tall'},
                         hoverinfo='skip'))

fig.show()